## OpenPredict examples

Example call to the OpenPredict Smart API accessible at https://openpredict.137.120.31.102.nip.io

In [62]:
import json 
import requests

data = json.dumps({
  "message": {
    "n_results": "42",
    "query_graph": {
      "edges": [
        {
          "id": "e00",
          "source_id": "n00",
          "target_id": "n01",
          "type": "treated_by"
        }
      ],
      "nodes": [
        {
          "curie": "DRUGBANK:DB00394",
          "id": "n00",
          "type": "drug"
        },
        {
          "id": "n01",
          "type": "disease"
        }
      ]
    },
    "query_options": {
      "has_confidence_level": "0.5"
    }
  }
})
headers = {'Content-type': 'application/json'}
predict_drugs = requests.post('https://openpredict.137.120.31.102.nip.io/query',
                     data=data, headers=headers)
print(json.dumps( predict_drugs.json(), indent = 2))

 {
          "kg_id": "OMIM:146255",
          "qg_id": "n01"
        }
      ]
    },
    {
      "edge_bindings": [
        {
          "kg_id": "e237",
          "qg_id": "e00"
        }
      ],
      "node_bindings": [
        {
          "kg_id": "DRUGBANK:DB00394",
          "qg_id": "n00"
        },
        {
          "kg_id": "OMIM:601042",
          "qg_id": "n01"
        }
      ]
    },
    {
      "edge_bindings": [
        {
          "kg_id": "e238",
          "qg_id": "e00"
        }
      ],
      "node_bindings": [
        {
          "kg_id": "DRUGBANK:DB00394",
          "qg_id": "n00"
        },
        {
          "kg_id": "OMIM:208750",
          "qg_id": "n01"
        }
      ]
    },
    {
      "edge_bindings": [
        {
          "kg_id": "e239",
          "qg_id": "e00"
        }
      ],
      "node_bindings": [
        {
          "kg_id": "DRUGBANK:DB00394",
          "qg_id": "n00"
        },
        {
          "kg_id": "OMIM:606703",
          "qg_i

In [59]:
import json 
import requests

predict_drugs = requests.get('https://openpredict.137.120.31.102.nip.io/predict',
                     params={'entity':"DRUGBANK:DB00394"})
print(json.dumps( predict_drugs.json(), indent = 2))

 },
    {
      "score": 0.04622291959003722,
      "source": {
        "id": "DRUGBANK:DB00394",
        "label": "beclomethasone dipropionate",
        "type": "drug"
      },
      "target": {
        "id": "OMIM:208750",
        "label": "ataxia, deafness, and cardiomyopathy",
        "type": "disease"
      }
    },
    {
      "score": 0.04595926083935443,
      "source": {
        "id": "DRUGBANK:DB00394",
        "label": "beclomethasone dipropionate",
        "type": "drug"
      },
      "target": {
        "id": "OMIM:606703",
        "label": "familial dyskinesia and facial myokymia",
        "type": "disease"
      }
    },
    {
      "score": 0.04522888227653884,
      "source": {
        "id": "DRUGBANK:DB00394",
        "label": "beclomethasone dipropionate",
        "type": "drug"
      },
      "target": {
        "id": "OMIM:206200",
        "label": "IRIDA syndrome",
        "type": "disease"
      }
    },
    {
      "score": 0.045173552721149215,
      "source":

In [60]:
# For a disease
predict_diseases = requests.get('https://openpredict.137.120.31.102.nip.io/v1/predict',
                     params={'entity':"OMIM:268220", 
                             'classifier':'Predict OMIM-DrugBank'})
print(json.dumps( predict_diseases.json(), indent = 2))

{
  "detail": "The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.",
  "status": 404,
  "title": "Not Found",
  "type": "about:blank"
}


## Translator identifiers resolution

Translator ID preferences:
* [For Disease](https://github.com/biolink/biolink-model/blob/master/biolink-model.yaml#L2853)
* [Chemical substance](https://github.com/biolink/biolink-model/blob/master/biolink-model.yaml#L3097) 
* [Drug](https://github.com/biolink/biolink-model/blob/master/biolink-model.yaml#L3119)

Resolve CURIEs using Translator API services: 

In [61]:
resolve_curies = requests.get('https://nodenormalization-sri.renci.org/get_normalized_nodes',
                     params={'curie':["OMIM:268220", "DRUGBANK:DB00394", "HP:0007354", "HGNC:613"]})
print( json.dumps( resolve_curies.json(), indent = 2))

{
  "OMIM:268220": {
    "id": {
      "identifier": "MONDO:0009994",
      "label": "alveolar rhabdomyosarcoma (disease)"
    },
    "equivalent_identifiers": [
      {
        "identifier": "MONDO:0009994",
        "label": "alveolar rhabdomyosarcoma (disease)"
      },
      {
        "identifier": "DOID:4051"
      },
      {
        "identifier": "OMIM:268220"
      },
      {
        "identifier": "ORPHANET:99756"
      },
      {
        "identifier": "UMLS:C0206655"
      },
      {
        "identifier": "MESH:D018232"
      },
      {
        "identifier": "MEDDRA:10065867"
      },
      {
        "identifier": "NCIT:C3749"
      },
      {
        "identifier": "SNOMEDCT:404053004"
      },
      {
        "identifier": "SNOMEDCT:63449009"
      },
      {
        "identifier": "HP:0006779",
        "label": "Alveolar rhabdomyosarcoma"
      }
    ],
    "type": [
      "disease",
      "named_thing",
      "biological_entity",
      "disease_or_phenotypic_feature"
    ]
  }